In [1]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from numpy import load
from torch.utils.data import Dataset

In [2]:
# load array
Data = load('Data_09_lob_dist.npy')
# print the array

In [3]:
Data=np.nan_to_num(Data)

In [4]:
Xdata=[]
Ydata=[]
Xtrain_data=[]
Ytrain_data=[]
Xtest_data=[]
Ytest_data=[]

for idx in range(len(Data)//39):
#     print(np.isinf(Data[39*idx:39*(idx)+38][:,:].tolist()).any())
    if(np.isinf(Data[39*idx:39*(idx)+38][:,:].tolist()).any()):
        print(np.isinf(Data[39*idx:39*(idx)+38][:,:].tolist()).any())
        continue
    if idx< 3735:
        Xtrain_data.append(Data[39*idx:39*(idx)+39][:,:].tolist())
        Ytrain_data.append(Data[39*idx:39*(idx)+39][:,1].tolist())
    else:
        Xtest_data.append(Data[39*idx:39*(idx)+39][:,:].tolist())
        Ytest_data.append(Data[39*idx:39*(idx)+39][:,1].tolist())


Xtrain_data=np.vstack(Xtrain_data)
Ytrain_data=np.vstack(Ytrain_data)
Xtest_data=np.vstack(Xtest_data)
Ytest_data=np.vstack(Ytest_data)

Xtrain_data=torch.FloatTensor(Xtrain_data)
Ytrain_data=torch.IntTensor(Ytrain_data)
Ytrain_data=Ytrain_data.view(-1)


Xtest_data=np.vstack(Xtest_data)
Ytest_data=np.vstack(Ytest_data)
Xtest_data=torch.FloatTensor(Xtest_data)
Ytest_data=torch.IntTensor(Ytest_data)
Ytest_data=Ytest_data.view(-1)

In [5]:
Ytrain_data=Ytrain_data[:,1:]>Ytrain_data[:,:-1]
Ytrain_data=torch.LongTensor(Ytrain_data)
Ytrain_data=torch.cat((Ytrain_data,torch.zeros(Ytrain_data.shape[0],1)+2),axis=1)
Ytrain_data=Ytrain_data.long()
Ytrain_data=Ytrain_data.view(-1)

Xtrain_data=torch.FloatTensor(Xtrain_data)


Ytest_data=Ytest_data[:,1:]>Ytest_data[:,:-1]
Ytest_data=torch.LongTensor(Ytest_data)
Ytest_data=torch.cat((Ytest_data,torch.zeros(Ytest_data.shape[0],1)+2),axis=1)

Xtest_data=np.vstack(Xtest_data)
Ytest_data=np.vstack(Ytest_data)
Xtest_data=torch.FloatTensor(Xtest_data)
Ytest_data=torch.LongTensor(Ytest_data)
Ytest_data=Ytest_data.view(-1)

In [6]:
744*39

29016

In [7]:
Ytrain_data.shape

torch.Size([145665])

In [8]:
Xtrain_data.shape

torch.Size([145665, 17])

In [9]:
Xtrain_data.size()

torch.Size([145665, 17])

Ytrain_data=2*(Ytrain_data>0).long()+(Ytrain_data==0).long()
Ytest_data=2*(Ytest_data>0).long()+(Ytest_data==0).long()

Ydata=(torch.stack([Ydata>0,Ydata==0, Ydata<0],axis=0)).float()

In [10]:
Ytrain_data=Ytrain_data.T
Ytest_data=Ytest_data.T

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# def batchify(data, bsz):
#     # 데이터셋을 bsz 파트들로 나눕니다.
#     nbatch = data.size(0) // bsz
#     # 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들) 은 잘라냅니다.
#     data = data.narrow(0, 0, nbatch * bsz)
#     # 데이터에 대하여 bsz 배치들로 동등하게 나눕니다.
#     data = data.view(bsz, -1,data.size(1)).transpose(0,1).contiguous()
#     return data.to(device)


def batchify(data, bsz,bptt):
    # 데이터셋을 bsz 파트들로 나눕니다.
    if(data.size(0)%(bsz*bptt)!=0):
        data=data.view(-1,bptt,data.size(1)).transpose(0,1).contiguous()
        return data.to(device)
    nbatch = data.size(0) // bsz
    # 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들) 은 잘라냅니다.
    data = data.narrow(0, 0, nbatch * bsz)
    # 데이터에 대하여 bsz 배치들로 동등하게 나눕니다.
    data = data.view(bsz, -1,data.size(1)).transpose(0,1).contiguous()
    return data.to(device)


# batch_size = 25
# eval_batch_size = 25

# Xtrain_data = batchify(Xtrain_data, batch_size)
# Xtest_data = batchify(Xtest_data, eval_batch_size)
# Ytrain_data = batchify(Ytrain_data, batch_size)
# Ytest_data = batchify(Ytest_data, eval_batch_size)

In [12]:
bptt = 39
def get_batch(source, i,bs):
    seq_len = min(bptt*bs, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [13]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

INPUT_SIZE=Xtrain_data.shape[1]
EMB_SIZE=64
HIDDEN_SIZE=64
NUM_LAYERS=6
PROJ_SIZE=3
BATCH_SIZE=16

In [14]:
class Seq2SeqLSTM(nn.Module):
    def __init__(self,
                 input_size: int,
                 hidden_size: int,
                 num_layers: int,
                 emb_size:int,
                 proj_size:int):
        
        super(Seq2SeqLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=emb_size,hidden_size=hidden_size,num_layers=num_layers)
        self.generator = nn.Linear(emb_size, proj_size)
        self.src_tok_emb = nn.Linear(input_size, emb_size)
        self.bn1=nn.BatchNorm1d(BATCH_SIZE)
    def forward(self,
                src: Tensor):

        src_emb = self.src_tok_emb(src)
        src_emb=self.bn1(src_emb)
#         outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
#                                 src_padding_mask, tgt_padding_mask, memory_key_padding_mask)


        outs,_ = self.lstm(src_emb)
#         print(outs.size())
        return self.generator(outs)

In [15]:
torch.manual_seed(0)
lstm=Seq2SeqLSTM(input_size=INPUT_SIZE,hidden_size=HIDDEN_SIZE,num_layers=NUM_LAYERS,emb_size=EMB_SIZE,proj_size=PROJ_SIZE)
for p in lstm.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
lstm = lstm.to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=1e-5, betas=(0.9, 0.98), eps=1e-9)

In [16]:
Xtrain_data[0]

tensor([1.2950e+04, 1.7992e-02, 1.9043e-02, 0.0000e+00, 5.0380e+04, 4.6628e+04,
        1.7019e+03, 1.9367e+03, 1.5534e+03, 1.3029e+03, 9.6608e+02, 9.7936e+02,
        8.9463e+02, 1.4255e+03, 2.6914e+03, 2.8420e+03, 0.0000e+00])

In [17]:
Xtrain_data.shape

torch.Size([145665, 17])

In [18]:
Xtest_data.shape

torch.Size([29055, 17])

In [19]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
#     for batch, i in enumerate(range(0, Xtrain_data.size(0) - 1, BATCH_SIZE*bptt)):
    i=0
    while(i<Xtest_data.size(0)):
        data, org_targets = get_batch(Xtrain_data, i,BATCH_SIZE)
        if (data.isnan().any() or data.isinf().any()):
#            print(data)
            continue
#         _,targets = get_batch(Ytrain_data,i)
        targets,_ = get_batch(Ytrain_data,i,BATCH_SIZE)
#         src = src.to(DEVICE)
#         tgt = tgt.to(DEVICE)
        targets=torch.unsqueeze(targets,1)
        src=batchify(data,BATCH_SIZE,bptt)
        tgt=batchify(targets,BATCH_SIZE,bptt)
        src=src[:]
        tgt=tgt[:]
        logits = model(src)

        optimizer.zero_grad()

        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
        
        _,predicted=torch.max(logits,-1)
        
        correct += (tgt.squeeze() == predicted).sum().item()
        total+=len(predicted)*BATCH_SIZE
        tot0+=(0== tgt.squeeze()).sum().item()
        tot1+=(1== tgt.squeeze()).sum().item()
        tot2+=(2== tgt.squeeze()).sum().item()
        correct0+=((0== predicted) &(0==tgt.squeeze())).sum().item()
        correct1+=((1== predicted)&(1==tgt.squeeze())).sum().item()
        correct2+=((2== predicted)&(2==tgt.squeeze())).sum().item()
        
        i+=targets.size()[0]

    print("TrainAcc:",correct/total*100)            
    print("TrainAcc0:",correct0/tot0*100,correct0,tot0)     
    print("TrainAcc1:",correct1/tot1*100,correct1,tot1)     
    print("TrainAcc2:",correct2/tot2*100,correct2,tot2)     
    return losses / Xtrain_data.size(0)


def evaluate(model):
    model.eval()
    losses = 0
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
    with torch.no_grad():
        
        i=0
        while(i<Xtest_data.size(0)):
#         for i in range(0, Xtest_data.size(0) - 1, bptt):

            data, org_targets = get_batch(Xtest_data, i,BATCH_SIZE)
            if (data.isnan().any() or data.isinf().any()):
                print(data)
                continue
            targets,_ = get_batch(Ytest_data,i,BATCH_SIZE)
            targets=torch.unsqueeze(targets,1)
            src=batchify(data,BATCH_SIZE,bptt)
            tgt=batchify(targets,BATCH_SIZE,bptt)
            print(src.shape)
            print(tgt.shape)
            logits = model(src)
#            print(logits.reshape(-1, logits.shape[-1]).shape,'AA')
#            print(tgt.reshape(-1).shape,'BB')
            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
            if(loss.isnan()):
                print(src,tgt_input)
                break
            losses += loss.item()
            _,predicted=torch.max(logits,-1)

            
            correct += (tgt.squeeze() == predicted).sum().item()
            total+=len(predicted)*BATCH_SIZE
            tot0+=(0== tgt.squeeze()).sum().item()
            tot1+=(1== tgt.squeeze()).sum().item()
            tot2+=(2== tgt.squeeze()).sum().item()
            correct0+=((0== predicted) &(0==tgt.squeeze())).sum().item()
            correct1+=((1== predicted)&(1==tgt.squeeze())).sum().item()
            correct2+=((2== predicted)&(2==tgt.squeeze())).sum().item()
            i+=targets.size()[0]
            
    print(total)
    print(correct)
    print("Acc:",correct/total*100)            
    print("Acc0:",correct0/tot0*100,correct0,tot0)     
    print("Acc1:",correct1/tot1*100,correct1,tot1)     
    print("Acc2:",correct2/tot2*100,correct2,tot2)     
    return losses / Xtest_data.size(0)

In [20]:
Xtest_data.shape

torch.Size([29055, 17])

In [21]:
Ytest_data.shape

torch.Size([29055])

In [22]:
evaluate(lstm)

torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 17])
torch.Size([39, 16, 1])
torch.Size([39, 16, 

RuntimeError: shape '[-1, 39, 17]' is invalid for input of size 5950

In [ ]:
optimizer = torch.optim.Adam(lstm.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-9)

In [ ]:
from timeit import default_timer as timer
NUM_EPOCHS = 1000
best_val_loss=100000000
for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(lstm, optimizer)
    end_time = timer()
    val_loss = evaluate(lstm)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = lstm
PATH='best_model_seq_mid_1'
torch.save(best_model.state_dict(), PATH)